## Time Transforms

Run this notebook in Vertex Workbench. In this notebook, we will use extracts from the timestamp
as input features.

In [1]:
%load_ext bigquery_magics

## Parts of time

Use extracted value from the timestamp

In [2]:
%%bigquery
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  EXTRACT(hour FROM dep_time) AS dep_hour,
  EXTRACT(dayofweek FROM dep_time) AS dep_day,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,ontime,dep_delay,taxi_out,distance,origin,dest,dep_hour,dep_day,is_eval_day
0,late,12.0,39.0,692.0,ABE,ATL,21,2,False
1,ontime,8.0,12.0,692.0,ABE,ATL,20,1,False
2,ontime,-6.0,10.0,692.0,ABE,ATL,15,6,False
3,late,-1.0,45.0,692.0,ABE,ATL,11,5,False
4,ontime,-3.0,11.0,692.0,ABE,ATL,15,2,False


In [3]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_airports_timefeatures
TRANSFORM(
   * EXCEPT(dep_time),
   EXTRACT(hour FROM dep_time) AS dep_hour,
   EXTRACT(dayofweek FROM dep_time) AS dep_day
)
OPTIONS(input_label_cols=['ontime'], 
        model_type='boosted_tree_classifier',
        data_split_method='custom',
        data_split_col='is_eval_day',
        l2_reg=2.5,
        max_tree_depth=10)

AS

SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  dep_time,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False

Query is running:   0%|          |

""


In [4]:
%%bigquery

WITH predictions AS (
SELECT 
  *
FROM ML.PREDICT(MODEL dsongcp.arr_delay_airports_timefeatures,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  dep_time,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),

stats AS (
SELECT 
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)

SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

Query is running:   0%|          |

Downloading:   0%|          |

,correct_cancel,total_noncancel,correct_noncancel,total_cancel,rmse
0,0.841317,3919102,0.965521,851562,0.204371


## Day of week as categorical

By default, BigQuery ML treats numbers as numeric. We want day of week to be categorical.
We can cast it as a string, or we can treat it as weekday/weekend

In [10]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_airports_timefeatures
TRANSFORM(
   * EXCEPT(dep_time),
   EXTRACT(hour FROM dep_time) AS dep_hour,
   IF(EXTRACT(dayofweek FROM dep_time) BETWEEN 2 and 6, 'weekday', 'weekend') AS dep_day
)
OPTIONS(input_label_cols=['ontime'], 
        model_type='boosted_tree_classifier',
        data_split_method='custom',
        data_split_col='is_eval_day',
        l2_reg=2.5,
        max_tree_depth=10)

AS

SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  dep_time,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False

Query is running:   0%|          |

""


In [7]:
%%bigquery

WITH predictions AS (
SELECT 
  *
FROM ML.PREDICT(MODEL dsongcp.arr_delay_airports_timefeatures,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  dep_time,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),

stats AS (
SELECT 
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)

SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

Query is running:   0%|          |

Downloading:   0%|          |

,correct_cancel,total_noncancel,correct_noncancel,total_cancel,rmse
0,0.841085,3920001,0.965478,851562,0.204236


## Feature Cross

Use the combination of hour and day at a specific airport.
bucketize the hour as well so that we don't have too many hours.

In [8]:
%%bigquery
CREATE OR REPLACE MODEL dsongcp.arr_delay_airports_timefeatures
TRANSFORM(
   * EXCEPT(dep_time),
   ML.FEATURE_CROSS(STRUCT(
     ML.BUCKETIZE(EXTRACT(hour FROM dep_time), [0, 4, 8, 12, 16, 20]) AS dep_hour,
     IF(EXTRACT(dayofweek FROM dep_time) BETWEEN 2 and 6, 
                'weekday', 'weekend') AS dep_day,
     origin
   )) AS day_hour
)
OPTIONS(input_label_cols=['ontime'], 
        model_type='boosted_tree_classifier',
        data_split_method='custom',
        data_split_col='is_eval_day',
        l2_reg=2.5,
        max_tree_depth=10
       )

AS

SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  dep_time,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False

Query is running:   0%|          |

""


In [9]:
%%bigquery

WITH predictions AS (
SELECT 
  *
FROM ML.PREDICT(MODEL dsongcp.arr_delay_airports_timefeatures,
                 (
SELECT
  IF(arr_delay < 15, 'ontime', 'late') AS ontime,
  dep_delay,
  taxi_out,
  distance,
  origin,
  dest,
  dep_time,
  IF(is_train_day = 'True', False, True) AS is_eval_day
FROM dsongcp.flights_tzcorr f
JOIN dsongcp.trainday t
ON f.FL_DATE = t.FL_DATE
WHERE
  f.CANCELLED = False AND 
  f.DIVERTED = False AND
  t.is_train_day = 'False'
                 ),
                 STRUCT(0.7 AS threshold))),

stats AS (
SELECT 
  COUNTIF(ontime != 'ontime' AND ontime = predicted_ontime) AS correct_cancel
  , COUNTIF(predicted_ontime = 'ontime') AS total_noncancel
  , COUNTIF(ontime = 'ontime' AND ontime = predicted_ontime) AS correct_noncancel
  , COUNTIF(ontime != 'ontime') AS total_cancel
  , SQRT(SUM((IF(ontime = 'ontime', 1, 0) - p.prob) * (IF(ontime = 'ontime', 1, 0) - p.prob))/COUNT(*)) AS rmse
FROM predictions, UNNEST(predicted_ontime_probs) p
WHERE p.label = 'ontime'
)

SELECT
   correct_cancel / total_cancel AS correct_cancel
   , total_noncancel
   , correct_noncancel / total_noncancel AS correct_noncancel
   , total_cancel
   , rmse
FROM stats

Query is running:   0%|          |

Downloading:   0%|          |

,correct_cancel,total_noncancel,correct_noncancel,total_cancel,rmse
0,0.841105,3919546,0.965478,851562,0.204503


Copyright 2021 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.